In [1]:
#필요한 라이브러리 불러옴
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn

from nltk.tokenize import word_tokenize

from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [2]:
#셀레니움을 이용해 Packt 사이트와 아마존에서 책 데이터들을 크롤링해옴

# Packt 사이트의 Book Data를 읽어오는 코드.

# Packt 사이트 메인으로 이동.
url = 'https://www.packtpub.com/'
chromedriver = 'C:/test/chromedriver.exe'
# 세부 카테고리 내용을 리스트에 저장.
category_list = ['Business_And_Other', 'Cloud_And_Networking','Data', 'GaneDevelopment','IoT_And_Hardware', 'Mobile','Programming', 'Tech','WebDevelopment']

options = webdriver.ChromeOptions()
options.add_argument('--start-fullscreen')

driver = webdriver.Chrome(chromedriver, options=options)

# 세부 카테고리별로 진행.
for cate_num in range(2, 11): 
    print(str(cate_num-1) + '번 카테고리 작업 시작.')
    first_list = []
    driver.get(url)

    # Packt 사이트의 All Book으로 접근.
    driver.find_element_by_xpath('//*[@id="desktop"]/div/div/div/a').click()
    driver.find_element_by_xpath('//*[@id="desktop"]/div/div/div/div/div/div/div/div/div[2]/div[2]/div[2]/a').click()
    sleep(3)
    # 한 Page 당 48개의 책 정보를 읽어오도록 변경.
    driver.find_element_by_xpath('//*[@id="hits-per-page-container"]/label[3]').click()
    sleep(3)
    # 세부 카테고리로 이동.
    text = '//*[@id="instant-search-facets-container"]/div[4]/div/div/div[2]/div/div[' + str(cate_num) + ']/div/div/label'
    driver.find_element_by_xpath(text).click()
    sleep(3)

    # 각 카테고리 별로 Page 수가 다른 문제 해결을 위해 기입.
    try:
        # 1Page에서 읽어온 Book Data를 list에 저장.
        notices = driver.find_elements_by_tag_name('h5')
        for n in notices:
            first_list.append(n.text)
        sleep(3)
        
        # 20 Page까지 반복.
        for num in range(20):
            # 다음 Page로 이동.
            driver.find_element_by_xpath('//*[@id="instant-search-pagination-container"]/div/ul/li[last()]/a').click()
            sleep(3)

            # 각 Page에서 읽어온 Book Data를 list에 저장.
            notices = driver.find_elements_by_tag_name('h5')
            for n in notices:
                first_list.append(n.text)
            sleep(3)
    except :
        print('페이지 부재 발생. 다음 카테고리로 이동.')
    
    # 읽어온 자료를 데이터 프레임으로 변환하여 카테고리 속성을 추가하여 xlsx 파일로 저장.
    print(str(cate_num-1) + '번 카테고리 저장 작업 시작.')
    df_data = pd.DataFrame(first_list)
    df_data[1] = category_list[cate_num-2]
    text = 'Packt/' + str(cate_num-1) + '_' + category_list[cate_num-2] + '.xlsx'
    df_data.to_excel(text)
    
# 각 카테고리 별 도서 데이터를 읽어오고 중복 요소를 삭제 후 병합.
print('Total 파일 생성 작업 시작.')
df1 = pd.read_excel('사용한 데이터 모음/Packt/1_Business_And_Other.xlsx')
df2 = pd.read_excel('사용한 데이터 모음/Packt/2_Cloud_And_Networking.xlsx')
df3 = pd.read_excel('사용한 데이터 모음/Packt/3_Data.xlsx')
df4 = pd.read_excel('사용한 데이터 모음/Packt/4_GaneDevelopment.xlsx')
df5 = pd.read_excel('사용한 데이터 모음/Packt/5_IoT_And_Hardware.xlsx')
df6 = pd.read_excel('사용한 데이터 모음/Packt/6_Mobile.xlsx')
df7 = pd.read_excel('사용한 데이터 모음/Packt/7_Programming.xlsx')
df8 = pd.read_excel('사용한 데이터 모음/Packt/8_Tech.xlsx')
df9 = pd.read_excel('사용한 데이터 모음/Packt/9_WebDevelopment.xlsx')

del df1['Unnamed: 0']
del df2['Unnamed: 0']
del df3['Unnamed: 0']
del df4['Unnamed: 0']
del df5['Unnamed: 0']
del df6['Unnamed: 0']
del df7['Unnamed: 0']
del df8['Unnamed: 0']
del df9['Unnamed: 0']

df_total = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])
df_total.to_excel(excel_writer='사용한 데이터 모음/Packt/0_AllBookName.xlsx')

print('전체 작업 종료.')

In [3]:
# Amazon의 사이트의 Book Data를 읽어오는 코드.

# Amazon 사이트 메인으로 이동.
url = 'https://www.amazon.com/s/ref=lp_283155_nr_n_7?fst=as%3Aoff&rh=n%3A283155%2Cn%3A%211000%2Cn%3A5&bbn=1000&ie=UTF8&qid=1603020080&rnid=1000'
chromedriver = 'C:/test/chromedriver.exe'

category_list = ['Programming', 'ComputerScience', 'Games', 'ProgrammingLanguages', 'Networking_And_Cloudcomputing', 'Database_And_BigData', 'Hardware_And_DIY', 'Graphics_And_Design', 'Software', 'Certification', 'WebDevelopment_And_Design', 'OperatingSystem']

options = webdriver.ChromeOptions()
options.add_argument('--start-fullscreen')

driver = webdriver.Chrome(chromedriver, options=options)

# LogIn 과정 진행.
driver.get(url)
driver.find_element_by_xpath('//*[@id="nav-belt"]/div[3]/div/a[2]').click()
sleep(3)
elem_login = driver.find_element_by_id("ap_email")
elem_login.clear()
elem_login.send_keys("zmfpdydgo@naver.com")
driver.find_element_by_xpath('//*[@class="a-box-inner a-padding-extra-large"]/div[2]/span').click()
elem_login = driver.find_element_by_id("ap_password")
elem_login.clear()
elem_login.send_keys("dos72013")
driver.find_element_by_xpath('//*[@class="a-box-inner a-padding-extra-large"]/form/div/div[2]/span').click()
sleep(3)

# 세부 카테고리 숫자만큼 반복.
for Cate_num in range(1, 13):
    
    print(str(Cate_num) + "번 카테고리 작업 시작.")
    
    if Cate_num == 3:
        continue

    first_title = []
    driver.get(url)
    sleep(3)
    
    # 각 카테고리 별로 이동.
    if Cate_num <= 6:
        text = '//*[@class="a-section bxw-categorytiles-dnav__row"][1]/div[' + str(Cate_num) + ']/a'
        driver.find_element_by_xpath('//*[@class="a-section bxw-categorytiles-dnav__row"][1]/div[' + str(Cate_num) + ']/a').click()
    else:
        text = '//*[@class="a-section bxw-categorytiles-dnav__row"][2]/div[' + str(Cate_num - 6) + ']/a'
        driver.find_element_by_xpath('//*[@class="a-section bxw-categorytiles-dnav__row"][2]/div[' + str(Cate_num - 6) + ']/a').click()
        
    Main_Cate_url = driver.current_url

    # 각 카테고리의 서브 카테고리 별로 작업을 수행.
    for Sub_Cate_num in range(1, 11):
        try:
            print(str(Cate_num) + "-" + str(Sub_Cate_num) + ' 작업 시작.')
            driver.get(Main_Cate_url)
            
            # 서브 카테고리 별로 이동.
            Cate_Button = '//*[@id="merchandised-content"]/div[3]/div[2]/ul/span[' + str(Sub_Cate_num) + ']/li'
            driver.find_element_by_xpath(Cate_Button).click()
            
            # 1Page 데이터 읽어오기.
            notices = driver.find_elements_by_tag_name('h2')
            for n in notices:
                first_title.append(n.text)
            sleep(3)
            driver.find_element_by_xpath('//*[@id="pagn"]/span[last()]/a/span[last()]').click()
    
            # 20 Page까지 반복.
            for num in range(20):
                try:
                    notices = driver.find_elements_by_tag_name('h2')
                    for n in notices:
                        first_title.append(n.text)
                    sleep(3)

                    driver.find_element_by_xpath('//*[@class="a-section a-spacing-none s-result-item s-flex-full-width s-widget"]/span/div/div/ul/li[last()]/a').click()
                    sleep(3)
                except :
                    print('페이지 부재 발생. 다음 세부 카테고리로 이동.')
        except :
            print('카테고리 부재 발생. 다음 카테고리로 이동.')
    
    # 읽어온 데이터에서 중복되는 요소들을 삭제.
    print(str(Cate_num) + "번 카테고리 중복 데이터 삭제 작업 시작.")
    new_title = []
    for li in first_title:
        if li not in new_title:
            new_title.append(li)
    new_title.remove('Need help?')

    # 읽어온 데이터를 저장.
    num = 0
    titles = []
    for n in new_title:
        titles.append(np.array(new_title[num]).flatten().tolist())
        num = num + 1

    # 읽어온 자료를 데이터 프레임으로 변환하여 카테고리 속성을 추가하여 xlsx 파일로 저장.
    print(str(Cate_num) + "번 카테고리 저장 작업 시작.")
    df_data = pd.DataFrame(titles)
    df_data[1] = category_list[Cate_num - 1]
    text = 'Amazon/' + str(Cate_num) + '_' + category_list[Cate_num-1] + '.xlsx'
    df_data.to_excel(text)
    
# 각 카테고리 별 도서 데이터를 읽어오고 중복 요소를 삭제 후 병합하는 코드.
print('Total 파일 생성 작업 시작.')
df1 = pd.read_excel('사용한 데이터 모음/Amazon/1_Programming.xlsx')
df2 = pd.read_excel('사용한 데이터 모음/Amazon/2_ComputerScience.xlsx')
df4 = pd.read_excel('사용한 데이터 모음/Amazon/4_ProgrammingLanguages.xlsx')
df5 = pd.read_excel('사용한 데이터 모음/Amazon/5_Networking_And_Cloudcomputing.xlsx')
df6 = pd.read_excel('사용한 데이터 모음/Amazon/6_Database_And_BigData.xlsx')
df7 = pd.read_excel('사용한 데이터 모음/Amazon/7_Hardware_And_DIY.xlsx')
df8 = pd.read_excel('사용한 데이터 모음/Amazon/8_Graphics_And_Design.xlsx')
df9 = pd.read_excel('사용한 데이터 모음/Amazon/9_Software.xlsx')
df10 = pd.read_excel('사용한 데이터 모음/Amazon/10_Certification.xlsx')
df11 = pd.read_excel('사용한 데이터 모음/Amazon/11_WebDevelopment_And_Design.xlsx')
df12 = pd.read_excel('사용한 데이터 모음/Amazon/12_OperatingSystem.xlsx')

del df1['Unnamed: 0']
del df2['Unnamed: 0']
del df4['Unnamed: 0']
del df5['Unnamed: 0']
del df6['Unnamed: 0']
del df7['Unnamed: 0']
del df8['Unnamed: 0']
del df9['Unnamed: 0']
del df10['Unnamed: 0']
del df11['Unnamed: 0']
del df12['Unnamed: 0']

df_total = pd.concat([df1, df2, df4, df5, df6, df7, df8, df9, df10, df11, df12])
df_total.to_excel(excel_writer='사용한 데이터 모음/Amazon/0_total.xlsx')

print('전체 작업 종료.')

In [4]:
# Packt 사이트의 2020년 BestSellers를 읽어오는 코드.

# Packt 사이트 메인으로 이동.
url = 'https://www.packtpub.com/'
chromedriver = 'C:/test/chromedriver.exe'
first_list = []

options = webdriver.ChromeOptions()
options.add_argument('--start-fullscreen')

driver = webdriver.Chrome(chromedriver, options=options)
driver.get(url)

# Packt 사이트의 2020년도 Best Sellers로 접근.
driver.find_element_by_xpath('//*[@id="desktop"]/div/div/div/a').click()
driver.find_element_by_xpath('//*[@id="desktop"]/div/div/div/div/div/div/div/div/div[2]/div[2]/div[4]/a').click()
sleep(3)
# 한 Page 당 48개의 책 정보를 읽어오도록 변경.
driver.find_element_by_xpath('//*[@id="hits-per-page-container"]/label[3]').click()
sleep(3)

# 1Page에서 읽어온 Book Data를 list에 저장.
notices = driver.find_elements_by_tag_name('h5')
for n in notices:
     first_list.append(n.text)
sleep(3)    

# 다음 페이지로 이동. 총 3회 반복.
for num in range(3):
    driver.find_element_by_xpath('//*[@id="instant-search-pagination-container"]/div/ul/li[last()]/a').click()
    sleep(3)

    # 각 Page에서 읽어온 Book Data를 list에 저장.
    notices = driver.find_elements_by_tag_name('h5')
    for n in notices:
        first_list.append(n.text)
    sleep(3)

# 읽어온 자료를 데이터 프레임으로 변환하여 xlsx 파일로 저장.
df_data = pd.DataFrame(first_list)
df_data.to_excel(excel_writer='사용한 데이터 모음/BestSellers/Packt_2020_Best.xlsx')

driver.close()

In [5]:
# Packt 사이트의 2019년 BestSellers를 읽어오는 코드.

# Packt 사이트 메인으로 이동.
url = 'https://www.packtpub.com/'
chromedriver = 'C:/test/chromedriver.exe'
first_list = []

options = webdriver.ChromeOptions()
options.add_argument('--start-fullscreen')

driver = webdriver.Chrome(chromedriver, options=options)
driver.get(url)

# Packt 사이트의 2019년도 Best Sellers로 접근.
driver.find_element_by_xpath('//*[@id="desktop"]/div/div/div/a').click()
driver.find_element_by_xpath('//*[@id="desktop"]/div/div/div/div/div/div/div/div/div[2]/div[2]/div[4]/a').click()
sleep(3)
driver.find_element_by_xpath('//*[@class="__cms-section __sub-categories"]/ul/li[3]/a[2]').click()
sleep(3)

# 한 Page 당 48개의 책 정보를 읽어오도록 변경.
driver.find_element_by_xpath('//*[@id="hits-per-page-container"]/label[3]').click()
sleep(3)

# 1Page에서 읽어온 Book Data를 list에 저장.
notices = driver.find_elements_by_tag_name('h5')
for n in notices:
     first_list.append(n.text)
sleep(3)    

# 읽어온 자료를 데이터 프레임으로 변환하여 xlsx 파일로 저장.
df_data = pd.DataFrame(first_list)
df_data.to_excel(excel_writer='사용한 데이터 모음/BestSellers/Packt_2019_Best.xlsx')

driver.close()

In [6]:
a = 5# Amazon의 2020년 BestSellers를 읽어오는 코드.

chromedriver = 'C:/test/chromedriver.exe'
url = 'https://www.amazon.com/s/ref=lp_283155_nr_n_7?fst=as%3Aoff&rh=n%3A283155%2Cn%3A%211000%2Cn%3A5&bbn=1000&ie=UTF8&qid=1603020080&rnid=1000'
first_title = []

options = webdriver.ChromeOptions()
options.add_argument('--start-fullscreen')

driver = webdriver.Chrome(chromedriver, options=options)
driver.get(url)

# BestSellers Category로 이동.
sleep(3)
driver.find_element_by_xpath('//*[@id="container"]/div/div[5]/h2/a').click()
    
# 1Page 데이터 읽어오기.
#notices = driver.find_element_by_class_name('p13n-sc-truncate-desktop-type2 p13n-sc-truncated')
notices = driver.find_elements_by_tag_name('a > div')
for n in notices:
     first_title.append(n.text)
sleep(3)

# 2Page로 이동하여 데이터 읽어오기.
driver.find_element_by_xpath('//*[@id="zg-center-div"]/div[2]/div/ul/li[last()]').click()
notices = driver.find_elements_by_tag_name('a > div')
for n in notices:
     first_title.append(n.text)
sleep(3)

# 읽어온 데이터에서 중복되는 요소들을 삭제.
new_title = []
for li in first_title:
    if li not in new_title:
        new_title.append(li)
new_title.remove('Hello, Sign in')
new_title.remove('0')
new_title.remove('Cart')
new_title.remove('')

# 읽어온 데이터를 DataFream으로 변환하여 카테고리 속성을 추가하고 CSV파일로 저장.
df_data = pd.DataFrame(new_title)
df_data.to_excel(excel_writer='사용한 데이터 모음/BestSellers/Amazon_2020_Best.xlsx')